In [1]:
import numpy
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.utils import np_utils


C:\Users\ajarun\AppData\Local\Continuum\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ajarun\AppData\Local\Continuum\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ajarun\AppData\Local\Continuum\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\U

In [2]:
data= pd.read_csv(r"C:\Users\ajarun\Documents\simplilearn\Capstone project\Ecommerce\train_data.csv")

In [3]:
import nltk
def tokenize_data(dataset):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    for i in range(dataset.shape[0]):
       dataset["reviews.text"][i] = tokenizer.tokenize(dataset["reviews.text"][i])
    return dataset


# In[80]:

newdata=tokenize_data(data)


# In[81]:

def remove_stop_words(dataset):
    stop_words = set(stopwords.words('english'))
    for i in range(dataset.shape[0]):
        dataset["reviews"][i] = ([token.lower() for token in dataset["reviews"][i] if token not in stop_words])
    
    return dataset


# In[82]:

def normalize(dataset):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    for i in range(dataset.shape[0]):
        dataset.reviews[i] = " ".join([lemmatizer.lemmatize(token) for token in dataset.reviews[i]]).strip()
    return dataset


# In[83]:

import string
def punct(dataset):
    table = str.maketrans('', '', string.punctuation)
    for i in range(dataset.shape[0]):
        dataset.reviews[i] = [w.translate(table) for w in dataset.reviews[i]]
    return dataset


# In[84]:

import re
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer


# In[85]:

vectorizer = TfidfVectorizer()
corpus = [] 
  
for i in range(0, len(data)): 
    text = re.sub('[^a-zA-Z]', '', str(data['reviews.text'][i])) 
    text = text.lower() 
    text = text.split() 
    ps = PorterStemmer() 
    text = ''.join(text) 
    corpus.append(text) 
  





In [4]:
# creating bag of words model 
X = vectorizer.fit_transform(corpus).toarray()
print(X.shape)
y = data['sentiment']  #.values 
num_labels = len(set(data['sentiment']))
X = X.astype('float32')
Y = y


(4000, 3594)


In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [6]:
from sklearn import model_selection
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,dummy_y,test_size = 0.1, random_state = 0)

In [7]:
input_dim = X.shape[1]

model = Sequential()
model.add(Dense(8, input_dim = input_dim , activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_x, train_y, epochs = 2, batch_size = 2)

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/2
3600/3600 [==============================] - 6878s 2s/step - loss: 0.3292 - accuracy: 0.9358
Epoch 2/2
400/400 [==============================] - 1s 2ms/step

accuracy: 94.25%


In [8]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import keras.utils
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout,SpatialDropout1D
import time

In [9]:
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim = input_dim, output_dim = 256))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(3, activation = 'softmax'))
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

C:\Users\ajarun\AppData\Local\Continuum\Anaconda3\envs\tf1\lib\site-packages\keras\layers\recurrent.py:2208: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


In [ ]:
history = model_lstm.fit(train_x, train_y,
    validation_split = 0.1,
    epochs = 2,
    batch_size = 512
)

Train on 3240 samples, validate on 360 samples
Epoch 1/8


In [ ]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()